## Deliverable 2. Create a Customer Travel Destinations Map.

In [10]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lata,PT,40.1629,-8.3327,61.81,71,89,5.91,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,44.71,70,75,3.44,broken clouds
2,2,Busselton,AU,-33.6500,115.3333,65.52,49,100,12.39,light rain
3,3,Rikitea,PF,-23.1203,-134.9692,73.51,84,100,9.93,light rain
4,4,Araguacu,BR,-12.9306,-49.8264,88.93,52,75,3.15,broken clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
new_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Busselton,AU,-33.6500,115.3333,65.52,49,100,12.39,light rain
3,3,Rikitea,PF,-23.1203,-134.9692,73.51,84,100,9.93,light rain
8,8,Georgetown,MY,5.4112,100.3354,82.33,89,20,0.00,few clouds
10,10,Saint-Pierre,RE,-21.3393,55.4781,74.88,74,40,10.36,scattered clouds
11,11,Carnarvon,AU,-24.8667,113.6333,66.27,94,2,5.75,clear sky


In [22]:
# 4a. Determine if there are any empty rows.
new_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_df = new_df.dropna()
new_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Busselton,AU,-33.6500,115.3333,65.52,49,100,12.39,light rain
3,3,Rikitea,PF,-23.1203,-134.9692,73.51,84,100,9.93,light rain
8,8,Georgetown,MY,5.4112,100.3354,82.33,89,20,0.00,few clouds
10,10,Saint-Pierre,RE,-21.3393,55.4781,74.88,74,40,10.36,scattered clouds
11,11,Carnarvon,AU,-24.8667,113.6333,66.27,94,2,5.75,clear sky


In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Busselton,AU,65.52,light rain,-33.6500,115.3333,
3,Rikitea,PF,73.51,light rain,-23.1203,-134.9692,
8,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
10,Saint-Pierre,RE,74.88,scattered clouds,-21.3393,55.4781,
11,Carnarvon,AU,66.27,clear sky,-24.8667,113.6333,
12,Cabo San Lucas,MX,81.70,clear sky,22.8909,-109.9124,
13,Liling,CN,73.83,overcast clouds,27.6667,113.5000,
14,Kunming,CN,66.96,clear sky,25.0389,102.7183,
15,Bathsheba,BB,84.81,broken clouds,13.2167,-59.5167,
16,Broome,US,77.70,clear sky,42.2506,-75.8330,


In [32]:
import numpy as np

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [31]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Busselton,AU,65.52,light rain,-33.6500,115.3333,Observatory Guest House
3,Rikitea,PF,73.51,light rain,-23.1203,-134.9692,People ThankYou
8,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
10,Saint-Pierre,RE,74.88,scattered clouds,-21.3393,55.4781,Lindsey Hôtel
11,Carnarvon,AU,66.27,clear sky,-24.8667,113.6333,Hospitality Carnarvon


In [34]:
# 8a. Create the output File (CSV)
output_file = "../Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))